In [1]:
import numpy as np
import os

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.scenario_setup import Scenario_simpy
from classes.combine import Combine

In [3]:
dir_output = os.path.join('output', 'sparky')

# Temporary rubbish dir extender to prevent overwriting:
while os.path.exists(dir_output):
    dir_output += '1'

os.mkdir(dir_output)

In [4]:
merge_dicts = dict(
    df_units = {},
    df_transfer = {},
    df_lsoa = {},
    df_selected_units = {},
    df_selected_transfer = {},
    df_selected_lsoa = {},
    results_summary_by_admitting_unit = {},
    results_summary_by_lsoa = {}
)

In [5]:
units_to_include = [
    'GL13NN',
    'SN36BB',
    'BS105NB',
    'EX314JB',
    'PL68DH',
    'TR13LQ',
    'EX25DW',
    'BA13NG',
    'SP28BJ',
    'TA15DA',
    'TQ27AA',
    'BS234TQ',
    'BA214AT',
]

## Scenario 1

In [6]:
scenario_name = 'drip-and-ship'

In [7]:
# Scenario overwrites default values
scenario = Scenario({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = scenario.get_unit_services()

df_units = df_units.reset_index()

mask = df_units['postcode'].isin(units_to_include)
df_units.loc[mask, 'selected'] = 1

df_units = df_units.set_index('postcode')

In [8]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
BA13NG,Royal United Hospital Bath,BA,Royal United Hospital Bath,1,0,0,nearest,E01014428,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
EX314JB,"North Devon District Hospital, Barnstaple",BP,North Devon District Hospital,1,0,0,nearest,E01020134,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
EX25DW,Royal Devon and Exeter Hospital,EX,Royal Devon and Exeter Hospital,1,0,0,nearest,E01020013,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
GL13NN,"Gloucestershire Royal Hospital, Gloucester",GL,Gloucestershire Royal Hospital,1,0,0,nearest,E01022311,NHS Gloucestershire ICB - 11M,E38000062,SICBL,England,NHS Gloucestershire Integrated Care Board,E54000043,"Gloucester, BSW, BNSSG and Somerset",1
SN36BB,"Great Western Hospital, Swindon",GW,Great Western Hospital Swindon,1,0,0,nearest,E01015583,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1
TA15DA,"Musgrove Park Hospital, Taunton",MP,Musgrove Park Hospital,1,0,0,nearest,E01029302,NHS Somerset ICB - 11X,E38000150,SICBL,England,NHS Somerset Integrated Care Board,E54000038,"Gloucester, BSW, BNSSG and Somerset",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1
SP28BJ,Salisbury District Hospital,SS,Salisbury District Hospital,1,0,0,nearest,E01032000,"NHS Bath and North East Somerset, Swindon and ...",E38000231,SICBL,England,"NHS Bath and North East Somerset, Swindon and ...",E54000040,"Gloucester, BSW, BNSSG and Somerset",1


In [9]:
dict_useful_data = scenario.process_scenario(df_units)

scenario_simpy = Scenario_simpy(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

Gather data for later combination:

In [10]:
# Update big dictionary of Dataframes:
merge_dicts['df_units'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa'][scenario_name] = dict_useful_data['df_lsoa']
merge_dicts['df_selected_units'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa

Save data to file:

In [11]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [12]:
file_dict = {
    'df_units': {
        'file': 'df_units.csv',
        'data': dict_useful_data['df_units']
    },
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': dict_useful_data['df_lsoa']
    },
    'df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': scenario_simpy.df_selected_units
    },
    'df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': scenario_simpy.df_selected_transfer
    },
    'df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': scenario_simpy.df_selected_lsoa
    },
    'results_summary_by_admitting_unit': {
        'file': 'results_summary_by_admitting_unit.csv',
        'data': model.results_summary_by_admitting_unit
    },
    'results_summary_by_lsoa': {
        'file': 'results_summary_by_lsoa.csv',
        'data': model.results_summary_by_lsoa
    },
}

In [13]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Scenario 2

In [14]:
scenario_name = 'mothership'

In [15]:
# Scenario overwrites default values
scenario = Scenario({
    # "setup": setup,
    'name': scenario_name,
    "run_duration": 5,  # years
})

df_units = scenario.get_unit_services()

df_units = df_units.reset_index()

# Set all non-MT units to have no IVT:
mask = df_units['use_mt'] == 0
df_units.loc[mask, 'use_ivt'] = 0

mask = (
    (df_units['postcode'].isin(units_to_include)) &
    (df_units['use_ivt'] == 1)
)
df_units.loc[mask, 'selected'] = 1


df_units = df_units.set_index('postcode')

In [16]:
df_units[df_units['selected'] == 1]

,stroke_team,short_code,ssnap_name,use_ivt,use_mt,use_msu,transfer_unit_postcode,lsoa_code,region,region_code,region_type,country,icb,icb_code,isdn,selected
postcode,,,,,,,,,,,,,,,,
PL68DH,"Derriford Hospital, Plymouth",DF,Derriford Hospital,1,1,1,nearest,E01015092,NHS Devon ICB - 15N,E38000230,SICBL,England,NHS Devon Integrated Care Board,E54000037,"Devon, Cornwall and Isles of Scilly",1
BS105NB,North Bristol Hospital (Southmead),NB,North Bristol Hospitals,1,1,1,nearest,E01014631,"NHS Bristol, North Somerset and South Gloucest...",E38000222,SICBL,England,"NHS Bristol, North Somerset and South Gloucest...",E54000039,"Gloucester, BSW, BNSSG and Somerset",1


In [17]:
dict_useful_data['df_transfer']

,transfer_unit_travel_time,transfer_unit_postcode,selected
postcode,,,
SY231ER,135.8,CF144XW,0
CB20QQ,0.0,CB20QQ,0
L97AL,0.0,L97AL,0
CH495PE,27.3,L97AL,0
BA13NG,33.7,BS105NB,1
...,...,...,...
SL24HL,NaN,<NA>,0
HP112TT,34.8,OX39DU,0
BA214AT,74.5,BS105NB,1


In [18]:
dict_useful_data = scenario.process_scenario(df_units)

scenario_simpy = Scenario_simpy(
    dict_useful_data['df_units'],
    dict_useful_data['df_transfer'],
    dict_useful_data['df_lsoa'],
)

model = Model(scenario_simpy)

model.run()

Gather data for later combination:

In [19]:
# Update big dictionary of Dataframes:
merge_dicts['df_units'][scenario_name] = dict_useful_data['df_units']
merge_dicts['df_transfer'][scenario_name] = dict_useful_data['df_transfer']
merge_dicts['df_lsoa'][scenario_name] = dict_useful_data['df_lsoa']
merge_dicts['df_selected_units'][scenario_name] = scenario_simpy.df_selected_units
merge_dicts['df_selected_transfer'][scenario_name] = scenario_simpy.df_selected_transfer
merge_dicts['df_selected_lsoa'][scenario_name] = scenario_simpy.df_selected_lsoa
merge_dicts['results_summary_by_admitting_unit'][scenario_name] = model.results_summary_by_admitting_unit
merge_dicts['results_summary_by_lsoa'][scenario_name] = model.results_summary_by_lsoa

Save data to file:

In [20]:
dir_output_scenario = os.path.join(dir_output, scenario_name)

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [21]:
file_dict = {
    'df_units': {
        'file': 'df_units.csv',
        'data': dict_useful_data['df_units']
    },
    'df_transfer': {
        'file': 'df_transfer.csv',
        'data': dict_useful_data['df_transfer']
    },
    'df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': dict_useful_data['df_lsoa']
    },
    'df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': scenario_simpy.df_selected_units
    },
    'df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': scenario_simpy.df_selected_transfer
    },
    'df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': scenario_simpy.df_selected_lsoa
    },
    'results_summary_by_admitting_unit': {
        'file': 'results_summary_by_admitting_unit.csv',
        'data': model.results_summary_by_admitting_unit
    },
    'results_summary_by_lsoa': {
        'file': 'results_summary_by_lsoa.csv',
        'data': model.results_summary_by_lsoa
    },
}

In [22]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)

## Combine multiple scenarios

In [23]:
combine = Combine()

combo_df_units = combine.combine_selected_units(
    merge_dicts['df_units'])

combo_df_transfer = combine.combine_selected_transfer(
    merge_dicts['df_transfer'])

combo_df_lsoa = combine.combine_selected_lsoa(
    merge_dicts['df_lsoa'])

combo_df_selected_units = combine.combine_selected_units(
    merge_dicts['df_selected_units'])

combo_df_selected_transfer = combine.combine_selected_transfer(
    merge_dicts['df_selected_transfer'])

combo_df_selected_lsoa = combine.combine_selected_lsoa(
    merge_dicts['df_selected_lsoa'])

combo_results_summary_by_admitting_unit = combine.combine_results_summary_by_admitting_unit(
    merge_dicts['results_summary_by_admitting_unit'])

combo_results_summary_by_lsoa = combine.combine_results_summary_by_lsoa(
    merge_dicts['results_summary_by_lsoa'])

Save data to file:

In [24]:
dir_output_scenario = os.path.join(dir_output, 'combined')

if os.path.exists(dir_output_scenario):
    pass
else:
    os.mkdir(dir_output_scenario)

In [25]:
file_dict = {
    'combo_df_units': {
        'file': 'df_units.csv',
        'data': combo_df_units,
    },
    'combo_df_transfer': {
        'file': 'df_transfer.csv',
        'data': combo_df_transfer,
    },
    'combo_df_lsoa': {
        'file': 'df_lsoa.csv',
        'data': combo_df_lsoa,
    },
    'combo_df_selected_units': {
        'file': 'df_selected_units.csv',
        'data': combo_df_selected_units, 
    },
    'combo_df_selected_transfer': {
        'file': 'df_selected_transfer.csv',
        'data': combo_df_selected_transfer,
    },
    'combo_df_selected_lsoa': {
        'file': 'df_selected_lsoa.csv',
        'data': combo_df_selected_lsoa,
    },
    'combo_results_summary_by_admitting_unit': {
        'file': 'results_summary_by_admitting_unit.csv',
        'data': combo_results_summary_by_admitting_unit,
    },
    'combo_results_summary_by_lsoa': {
        'file': 'results_summary_by_lsoa.csv',
        'data': combo_results_summary_by_lsoa,
    },
}

In [26]:
for data_type, data_dict in file_dict.items():
    path_to_file = os.path.join(dir_output_scenario, data_dict['file'])
    data_dict['data'].to_csv(path_to_file)